In [24]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os
import pandas as pd
import numpy as np

# --- 1. Configuration ---
api = KaggleApi()
api.authenticate()
dataset_slug = 'wordsforthewise/lending-club'

# The exact filename we want to download
file_to_download = 'accepted_2007_to_2018Q4.csv.gz'

# Where we want to save the file (in the current 'notebooks' folder)
output_path = '.' # '.' means the current directory

# --- 2. Download the Specific File ---
print(f"Starting download of '{file_to_download}'...")
print("This may take a few minutes...")

try:
    # Check if the uncompressed file already exists to avoid re-downloading
    if not os.path.exists('accepted_2007_to_2018Q4.csv'):
        api.dataset_download_file(dataset_slug,
                                  file_name=file_to_download,
                                  path=output_path,
                                  quiet=False) # quiet=False shows the progress bar
        
        # The Kaggle API downloads the .gz file, which Pandas can read directly.
        print("\nDownload complete.")
    else:
        print("\nMain data file already exists. Skipping download.")

except Exception as e:
    print(f"\nAn error occurred during download: {e}")
    raise

# --- 3. Optimized Loading (Reading the .gz file directly) ---
# Now, the final step: load the data using 'usecols'.
# Pandas is smart and can decompress the .gz file in real-time.

relevant_columns = [
    'loan_amnt', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_length',
    'home_ownership', 'annual_inc', 'verification_status', 'loan_status',
    'purpose', 'addr_state', 'dti', 'issue_d', 'earliest_cr_line'
]

print("\nStarting optimized loading of the dataset...")
try:
    # We use the name of the downloaded compressed file.
    # Pandas reads the compressed file directly!
    df = pd.read_csv('accepted_2007_to_2018Q4.csv.gz', 
                     usecols=relevant_columns, 
                     compression='gzip', # Tells Pandas it's a gzip file
                     low_memory=False)
    
    print(f"SUCCESS! Loaded {len(df)} rows and {len(df.columns)} columns.")
    df.info()

except Exception as e:
    print(f"\nAn error occurred during loading: {e}")
    raise


Starting download of 'accepted_2007_to_2018Q4.csv.gz'...
This may take a few minutes...
Dataset URL: https://www.kaggle.com/datasets/wordsforthewise/lending-club
accepted_2007_to_2018Q4.csv.gz: Skipping, found more recently modified local copy (use --force to force download)

Download complete.

Starting optimized loading of the dataset...
SUCCESS! Loaded 2260701 rows and 15 columns.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 15 columns):
 #   Column               Dtype  
---  ------               -----  
 0   loan_amnt            float64
 1   term                 object 
 2   int_rate             float64
 3   grade                object 
 4   sub_grade            object 
 5   emp_length           object 
 6   home_ownership       object 
 7   annual_inc           float64
 8   verification_status  object 
 9   issue_d              object 
 10  loan_status          object 
 11  purpose              object 
 12  addr_state         

In [25]:
# Map detailed loan statuses to simple 'Good' or 'Bad' categories for analysis
def map_loan_status(status):
    # We consider 'Fully Paid' and 'Current' loans as "Good"
    if status in ['Fully Paid', 'Current', 'Does not meet the credit policy. Status:Fully Paid']:
        return 'Good'
    # All other statuses for accepted loans that were not fully paid are considered "Bad" for risk analysis
    elif status in ['Charged Off', 'Default', 'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)', 'Does not meet the credit policy. Status:Charged Off']:
        return 'Bad'
    return 'Other' # Status that don't fit (rare at this stage)

# Apply the function to create the new target column
df['loan_condition'] = df['loan_status'].apply(map_loan_status)

# Filter to keep only loans with a defined final outcome (Good or Bad)
# and create a copy to avoid SettingWithCopyWarning
df_final = df[df['loan_condition'].isin(['Good', 'Bad'])].copy()

print("Count of 'Good' vs 'Bad' loans:")
print(df_final['loan_condition'].value_counts())


Count of 'Good' vs 'Bad' loans:
loan_condition
Good    1957056
Bad      303612
Name: count, dtype: int64


In [26]:
print("\nCleaning 'term' and 'emp_length' columns...")

# --- ROBUST CLEANING OF 'term' COLUMN ---
# Step 1: Ensure all values in the 'term' column are strings, handling nulls.
df_final['term'] = df_final['term'].astype(str)

# Step 2: Now, safely remove the text and convert to integer.
df_final['term'] = df_final['term'].str.replace(' months', '').astype(int)
print("'term' column cleaned successfully.")


# --- ROBUST CLEANING OF 'emp_length' COLUMN ---
# Step 1: Ensure all values in the 'emp_length' column are strings, handling nulls.
df_final['emp_length'] = df_final['emp_length'].astype(str)

# Step 2: Now, safely perform the text replacements.
df_final['emp_length'] = df_final['emp_length'].str.replace('< 1 year', '0')
df_final['emp_length'] = df_final['emp_length'].str.replace(' years', '')
df_final['emp_length'] = df_final['emp_length'].str.replace(' year', '')
df_final['emp_length'] = df_final['emp_length'].str.replace('+', '')
df_final['emp_length'] = df_final['emp_length'].str.replace('nan', '0') # Handles nulls that became the string 'nan'

# Step 3: Convert the cleaned column to a numeric type.
df_final['emp_length'] = pd.to_numeric(df_final['emp_length'])
print("'emp_length' column cleaned successfully.")


print("\nCleaning complete.")
print("\nUnique values for 'term':", sorted(df_final['term'].unique()))
print("Unique values for 'emp_length':", sorted(df_final['emp_length'].unique()))



Cleaning 'term' and 'emp_length' columns...
'term' column cleaned successfully.
'emp_length' column cleaned successfully.

Cleaning complete.

Unique values for 'term': [np.int64(36), np.int64(60)]
Unique values for 'emp_length': [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]


In [27]:
print("\nConverting 'issue_d' column to datetime format...")

# Convert the loan issue date column
df_final['issue_d'] = pd.to_datetime(df_final['issue_d'], format='%b-%Y')

print("Date conversion complete.")
print(df_final[['issue_d']].head())



Converting 'issue_d' column to datetime format...
Date conversion complete.
     issue_d
0 2015-12-01
1 2015-12-01
2 2015-12-01
3 2015-12-01
4 2015-12-01


In [28]:
# --- FINAL DATA TYPE ADJUSTMENT AND VALIDATION CELL ---

print("\nStarting final data type adjustments...")

# 1. Convert 'loan_amnt' to integer, if possible.
# We use 'Int64' (with a capital 'I'), a special Pandas type
# that supports integers even with null values (not our case, but it's good practice).
try:
    df_final['loan_amnt'] = df_final['loan_amnt'].astype('Int64')
    print("- 'loan_amnt' column converted to Integer.")
except Exception as e:
    print(f"- WARNING: Could not convert 'loan_amnt' to integer. Error: {e}")


# 2. Round other float columns to a reasonable number of decimal places.
# This improves their appearance in Tableau.
df_final['int_rate'] = df_final['int_rate'].round(2)
df_final['annual_inc'] = df_final['annual_inc'].round(2)
df_final['dti'] = df_final['dti'].round(4)
print("- Float columns rounded for better display.")


# --- Final Quick Validation ---
print("\n--- Checking final data types before saving ---")
print(df_final.info())

print("\n--- Checking final descriptive statistics ---")
print(df_final[['loan_amnt', 'int_rate', 'annual_inc', 'dti']].describe())




Starting final data type adjustments...
- 'loan_amnt' column converted to Integer.
- Float columns rounded for better display.

--- Checking final data types before saving ---
<class 'pandas.core.frame.DataFrame'>
Index: 2260668 entries, 0 to 2260698
Data columns (total 16 columns):
 #   Column               Dtype         
---  ------               -----         
 0   loan_amnt            Int64         
 1   term                 int64         
 2   int_rate             float64       
 3   grade                object        
 4   sub_grade            object        
 5   emp_length           int64         
 6   home_ownership       object        
 7   annual_inc           float64       
 8   verification_status  object        
 9   issue_d              datetime64[ns]
 10  loan_status          object        
 11  purpose              object        
 12  addr_state           object        
 13  dti                  float64       
 14  earliest_cr_line     object        
 15  loan_conditio

In [29]:
# --- FINAL SAVING CELL ---

# Define the output path to the 'data' folder
output_folder = '../data' 
output_filename = 'cleaned_loan_data.csv'
output_path = os.path.join(output_folder, output_filename)

# Ensure the target directory exists
os.makedirs(output_folder, exist_ok=True)

# Save the final, cleaned dataframe
# We explicitly set sep=',' to force comma as the separator
# and encoding='utf-8' to ensure universal character compatibility.
df_final.to_csv(output_path, index=False, sep=',', encoding='utf-8')

print(f"\nSUCCESS! File '{output_filename}' was created in the '{output_folder}' folder.")
print("The file was saved with a comma (,) as the separator and UTF-8 encoding.")



SUCCESS! File 'cleaned_loan_data.csv' was created in the '../data' folder.
The file was saved with a comma (,) as the separator and UTF-8 encoding.


In [30]:
# --- FINAL VALIDATION CELL ---
# Let's load the CSV file we just saved to ensure
# it was written and can be read correctly.

print("\n--- STARTING VALIDATION OF THE FINAL 'cleaned_loan_data.csv' FILE ---")

try:
    # Load the final file for validation
    df_validation = pd.read_csv('../data/cleaned_loan_data.csv')
    print("File 'cleaned_loan_data.csv' loaded successfully for validation.\n")

    # --- 1. Validate Structure and Data Types ---
    print("1. STRUCTURE AND DATA TYPES (dtypes):")
    print(df_validation.info())
    print("-" * 50)

    # --- 2. Validate Unique Values for Categorical Columns ---
    print("\n2. UNIQUE VALUES IN KEY CATEGORICAL COLUMNS:")
    
    categorical_columns_to_validate = [
        'term', 'grade', 'emp_length', 'home_ownership', 
        'verification_status', 'loan_condition', 'purpose'
    ]
    
    for col in categorical_columns_to_validate:
        print(f"\n   Unique values for '{col}':")
        # We use .unique() to see all possible values
        print(f"   {df_validation[col].unique()}")
    print("-" * 50)

    # --- 3. Validate Range and Statistics for Numeric Columns ---
    print("\n3. DESCRIPTIVE STATISTICS FOR NUMERIC COLUMNS:")
    # .describe() gives us count, mean, std, min, max, and quartiles.
    # This is great for identifying outliers or scaling issues.
    numeric_columns_to_validate = [
        'loan_amnt', 'int_rate', 'annual_inc', 'dti', 'emp_length'
    ]
    print(df_validation[numeric_columns_to_validate].describe())
    print("-" * 50)
    
    # --- 4. Check for Null Values ---
    print("\n4. COUNT OF NULL VALUES PER COLUMN:")
    print(df_validation.isnull().sum())
    print("-" * 50)
    
    print("\n--- VALIDATION COMPLETE ---")

except FileNotFoundError:
    print("ERROR: The file 'cleaned_loan_data.csv' was not found in the '../data' folder.")
except Exception as e:
    print(f"An error occurred during validation: {e}")


--- STARTING VALIDATION OF THE FINAL 'cleaned_loan_data.csv' FILE ---
File 'cleaned_loan_data.csv' loaded successfully for validation.

1. STRUCTURE AND DATA TYPES (dtypes):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260668 entries, 0 to 2260667
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   loan_amnt            int64  
 1   term                 int64  
 2   int_rate             float64
 3   grade                object 
 4   sub_grade            object 
 5   emp_length           int64  
 6   home_ownership       object 
 7   annual_inc           float64
 8   verification_status  object 
 9   issue_d              object 
 10  loan_status          object 
 11  purpose              object 
 12  addr_state           object 
 13  dti                  float64
 14  earliest_cr_line     object 
 15  loan_condition       object 
dtypes: float64(3), int64(3), object(10)
memory usage: 276.0+ MB
None
------------------------------